In [1]:
# 구글 드라이브와 연결
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 경로 설정
path = '/content/gdrive/MyDrive/pytest/data/eng-kor/'
!ls '/content/gdrive/MyDrive/pytest/data/eng-kor/'

eng-kor_small.txt  eng-kor.txt


In [3]:
# 데이터 확인
# 빠른 진행을 위해 small 데이터를 사용한다.
import pandas as pd
data = pd.read_csv(path+'eng-kor_small.txt', names=['source', 'target'], sep='\t', encoding='utf-8')
print('data length:', len(data))
print('data type:', type(data))
print('data shape:', data.shape)
print('data sample:\n', data.sample(5))

data length: 1000
data type: <class 'pandas.core.frame.DataFrame'>
data shape: (1000, 2)
data sample:
                   source              target
442        Nobody asked.         아무도 안 물어봤어.
971  Tom made me a cake.  톰이 날 위해 케이크를 만들었어.
228          How tragic!              너무 슬퍼!
491       Any questions?              질문 있어?
610      Tom hates cats.        톰은 고양이를 싫어해.


In [4]:
print('data.target length:', len(data.target))
print('data.target type:', type(data.target))
print('data.target shape:', data.target.shape)
print('data.target sample:\n', data.target.sample(5))

data.target length: 1000
data.target type: <class 'pandas.core.series.Series'>
data.target shape: (1000,)
data.target sample:
 504            잠이 와.
369    톰이 앓는 소리를 했어.
212          천천히 먹어.
98       톰이 거짓말을 했어.
544       저거 내 고양이야.
Name: target, dtype: object


In [5]:
# 시작부호와 종료부호 부착
# 데이터가 모두 3종이 필요하다. source 언어에는 encoder_input 1개, target 언어에는 decoder_input, decoder_target 2개이다
# encoder는 source 언어를 그대로 사용하면 되나, decoder는 seq2seq의 사용을 위해 \t, \n를 부착해야 한다
# decoder_input 데이터의 시작에는 \t, 문장의 끝에는 \n를 부착한다
# decoder_target 데이터는 \n만 필요하다
# tab을 받았기 때문에 자료가 더 오른쪽에서 출력되는 것을 알 수 있다
data.target_input = data.target.apply(lambda x : '\t'+x+'\n')
data.target_target = data.target.apply(lambda x : x+'\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [6]:
# 타입 확인
print('data.target_input length:', len(data.target_input))          
print('data.target_input type:', type(data.target_input))           
print('data.target_input shape:', data.target_input.shape)          
print('data.target_input sample:\n', data.target_input.sample(5))

data.target_input length: 1000
data.target_input type: <class 'pandas.core.series.Series'>
data.target_input shape: (1000,)
data.target_input sample:
 157          \t거기 앉아.\n
347    \t그 사람들은 복종했어.\n
85              \t가자!\n
219       \t우릴 용서해 줘.\n
368      \t톰이 씨익 웃었어.\n
Name: target, dtype: object


In [7]:
# 문장의 길이 maxlen 설정하기
# source와 target 문장의 최대 길이를 구한다
max_src_len = data.source.apply(lambda x: len(x)).max()
print('source sentence max length: ', max_src_len)               # source 문장의 최대 음절 길이로 maxlen을 설정
max_tar_len = data.target_input.apply(lambda x: len(x)).max()
print('target sentence max length: ', max_tar_len)               # target 문장의 최대 음절 길이로 maxlen을 설정

source sentence max length:  20
target sentence max length:  21


In [8]:
# Data Tokenizing
# 각 문자 종류에 대하여 숫자값을 배당한다
from keras.preprocessing.text import Tokenizer

# source 언어 Tokenizing
tokenizer_source = Tokenizer(num_words=None, char_level=True, lower=False)      # Tokenizer 객체 생성
tokenizer_source.fit_on_texts(data.source)     	                                # 인덱스를 구축한다
word_index_source = tokenizer_source.word_index                                 # 글자와 인덱스의 쌍을 가져온다

print('\n전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index_source))
print('word_index_source: ', word_index_source)


전체에서 64개의 고유한 토큰을 찾았습니다.
word_index_source:  {' ': 1, 'e': 2, 'o': 3, '.': 4, 'a': 5, 't': 6, 'i': 7, 's': 8, 'n': 9, 'r': 10, 'l': 11, 'd': 12, 'm': 13, 'h': 14, 'y': 15, 'u': 16, 'T': 17, 'g': 18, 'I': 19, 'c': 20, 'p': 21, 'w': 22, 'k': 23, "'": 24, 'v': 25, 'b': 26, 'f': 27, '?': 28, 'S': 29, '!': 30, 'W': 31, 'H': 32, 'C': 33, 'D': 34, 'E': 35, 'K': 36, 'A': 37, 'G': 38, 'Y': 39, 'N': 40, 'x': 41, 'F': 42, 'B': 43, 'L': 44, 'M': 45, 'q': 46, ',': 47, 'P': 48, 'R': 49, 'O': 50, 'z': 51, 'J': 52, 'j': 53, 'Q': 54, '-': 55, '7': 56, ':': 57, '4': 58, '5': 59, 'U': 60, '2': 61, '0': 62, '1': 63, '3': 64}


In [9]:
# target 언어 Tokenizing
# target 언어로는 target_input으로 하나만 만들면 된다
tokenizer_target = Tokenizer(num_words=None, char_level=True, lower=False)      # Tokenizer 객체 생성
tokenizer_target.fit_on_texts(data.target_input)     	                          # 인덱스를 구축한다
word_index_target = tokenizer_target.word_index                                 # 글자와 인덱스의 쌍을 가져온다

print('\n전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index_target))
print('word_index_target: ', word_index_target)


전체에서 558개의 고유한 토큰을 찾았습니다.
word_index_target:  {' ': 1, '\t': 2, '\n': 3, '.': 4, '어': 5, '이': 6, '톰': 7, '해': 8, '아': 9, '은': 10, '다': 11, '그': 12, '가': 13, '는': 14, '?': 15, '나': 16, '거': 17, '!': 18, '고': 19, '하': 20, '을': 21, '했': 22, '요': 23, '있': 24, '야': 25, '지': 26, '었': 27, '사': 28, '난': 29, '말': 30, '들': 31, '기': 32, '게': 33, '리': 34, '를': 35, '니': 36, '도': 37, '우': 38, '좋': 39, '와': 40, '내': 41, '에': 42, '람': 43, '무': 44, '자': 45, '마': 46, '서': 47, '봐': 48, '한': 49, '계': 50, '안': 51, '네': 52, '시': 53, '속': 54, '너': 55, '수': 56, '모': 57, '만': 58, '짓': 59, '라': 60, '두': 61, '누': 62, '일': 63, '세': 64, '정': 65, '웃': 66, '로': 67, '않': 68, '줘': 69, '았': 70, '제': 71, '렸': 72, '걸': 73, '없': 74, '려': 75, '물': 76, '미': 77, '저': 78, '여': 79, '건': 80, '죽': 81, '으': 82, '워': 83, '조': 84, '주': 85, '린': 86, '신': 87, '것': 88, '의': 89, '져': 90, '심': 91, '좀': 92, '운': 93, '인': 94, '구': 95, '진': 96, '렇': 97, '날': 98, '입': 99, '래': 100, '울': 101, '전': 102, '빨': 103, '피': 104, '히': 105, '할': 106

In [10]:
# Data Sequencing
# 배당된 숫자를 이용하여 각 문장의 문자를 숫자로 치환한다
# source 언어 Sequencing
# 문제가 있으면 오른쪽과 같이 시작한다. encoder_input = tokenizer_source.texts_to_sequences(list(data.source)) 
encoder_input = tokenizer_source.texts_to_sequences(data.source) 

print('\nResult of encoder_input sequencing: ')
print(data.source[0], encoder_input[0])
print(data.source[1], encoder_input[1])
print(data.source[2], encoder_input[2])
print(data.source[3], encoder_input[3])


Result of encoder_input sequencing: 
Go. [38, 3, 4]
Hi. [32, 7, 4]
Run! [49, 16, 9, 30]
Run. [49, 16, 9, 4]


In [11]:
# target 언어 Sequencing
decoder_input = tokenizer_target.texts_to_sequences(data.target_input)
decoder_target = tokenizer_target.texts_to_sequences(data.target_target)

print('\nResult of decoder_input sequencing: ')
print(data.target_input[0], decoder_input[0])
print(data.target_input[1], decoder_input[1])
print(data.target_input[2], decoder_input[2])

print('\nResult of decoder_target sequencing: ')
print(data.target_target[0], decoder_target[0])
print(data.target_target[1], decoder_target[1])
print(data.target_target[2], decoder_target[2])


Result of decoder_input sequencing: 
	가.
 [2, 13, 4, 3]
	안녕.
 [2, 51, 223, 4, 3]
	뛰어!
 [2, 272, 5, 18, 3]

Result of decoder_target sequencing: 
가.
 [13, 4, 3]
안녕.
 [51, 223, 4, 3]
뛰어!
 [272, 5, 18, 3]


In [12]:
# 타입 확인
print('data.source type:', type(data.source))              # Series
print('encoder_input type:', type(encoder_input))          # list

print('data.source:\n', data.source)
print('encoder_input\n:', encoder_input)

data.source type: <class 'pandas.core.series.Series'>
encoder_input type: <class 'list'>
data.source:
 0                       Go.
1                       Hi.
2                      Run!
3                      Run.
4                      Who?
               ...         
995    Do you like singing?
996    Do you like singing?
997    Don't you like cats?
998    Dreams do come true.
999    Everybody loves her.
Name: source, Length: 1000, dtype: object
encoder_input
: [[38, 3, 4], [32, 7, 4], [49, 16, 9, 30], [49, 16, 9, 4], [31, 14, 3, 28], [31, 3, 22, 30], [42, 7, 10, 2, 30], [32, 2, 11, 21, 30], [52, 16, 13, 21, 30], [52, 16, 13, 21, 4], [31, 5, 7, 6, 30], [31, 5, 7, 6, 30], [31, 5, 7, 6, 4], [43, 2, 18, 7, 9, 4], [32, 2, 11, 11, 3, 30], [19, 1, 8, 2, 2, 4], [19, 1, 6, 10, 15, 4], [19, 1, 22, 3, 9, 30], [50, 14, 1, 9, 3, 30], [49, 2, 11, 5, 41, 4], [29, 14, 3, 3, 6, 30], [29, 13, 7, 11, 2, 4], [37, 6, 6, 5, 20, 23, 30], [37, 6, 6, 5, 20, 23, 30], [42, 10, 2, 2, 51, 2, 30], [38, 2, 6, 1,

In [13]:
# Data Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

print('\nPadding result sample: ')
print(data.target_input[0], decoder_input[0])


Padding result sample: 
	가.
 [ 2 13  4  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [14]:
# 타입 확인
print('decoder_input length:', len(decoder_input)) 
print('decoder_input type:', type(decoder_input)) 
print('decoder_input shape:', decoder_input.shape) 

decoder_input length: 1000
decoder_input type: <class 'numpy.ndarray'>
decoder_input shape: (1000, 21)


In [15]:
# One-Hot-Encoding
# 케라스 원-핫 인코딩을 수행한다.
# 클래스의 수는 1을 올려주어야 한다(Padding으로 생긴 0을 추가로 받아야 함)
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input, num_classes=len(word_index_source)+1)
decoder_input = to_categorical(decoder_input, num_classes=len(word_index_target)+1)
decoder_target = to_categorical(decoder_target, num_classes=len(word_index_target)+1)

print('\nResult of One-Hot Encodded decoder_input sequencing: ')
print(decoder_input.shape)
print(data.target_input[0], decoder_input[0])


Result of One-Hot Encodded decoder_input sequencing: 
(1000, 21, 559)
	가.
 [[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [16]:
print('0-0\n', decoder_input[0][0])
print('\n0-1\n', decoder_input[0][1])
print('\n0-2\n', decoder_input[0][2])
print('\n0-3\n', decoder_input[0][3])
print('\n0-18\n', decoder_input[0][18])

0-0
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [17]:
# 타입 확인
print('decoder_input length:', len(decoder_input))
print('decoder_input type:', type(decoder_input))
print('decoder_input shape:', decoder_input.shape)

decoder_input length: 1000
decoder_input type: <class 'numpy.ndarray'>
decoder_input shape: (1000, 21, 559)


In [18]:
# 교사 강요를 이용한 모델 훈련
# 예측 과정에서는 이전 시점의 decoder_input의 예측 결과를 현재 시점의 decoder_input으로 넣을 것이다
# 그러나 훈련 과정에서는 그렇게 하면 잘못 예측된 이전 시점의 결과가 현재 시점으로 들어가게 된다
# 따라서 훈련 과정에서는 이전 시점의 decoder_input의 실제값을 현재 시점의 decoder_input으로 넣을 것이다
# 이를 교사 강요라고 한다
# 현재 Input이 3D이므로 Embedding으로 차원을 늘려주지 않아도 바로 3D input shape을 갖는 순환신경망을 이용할 수 있다

# Context Vector 만들기
# 훈련용 Encoder
from keras.models import Model
from keras.layers import Input, LSTM, Dense

encoder_inputs = Input(shape=(None, len(word_index_source)+1))      # 입력문의 길이가 문장마다 다르므로 None. 출력되는 내용은 최대 len(word_index_source)+1
encoder_lstm = LSTM(units=256, return_state=True)                   # encoder 내부 상태를 decoder로 넘겨주기 위해 return_state=True
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)    # return_state=True로 만들어진 모델이므로 은닉상태와 셀상태를 받는다. encoder_outputs는 사용하지 않는다
encoder_states = [state_h, state_c]                                 # 은닉상태와 셀상태를 받는다

In [19]:
# 훈련용 Decoder
decoder_inputs = Input(shape=(None, len(word_index_target)+1))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)            # 256 unit으로 된 encoder_states를 받아야 하므로 decoder의 은닉 상태도 256으로 동일하게 맞춰준다
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)  # decoder의 첫 상태를 encoder의 은닉 상태와 셀 상태로 한다
decoder_dense = Dense(len(word_index_target)+1, activation='softmax')               # decoder의 은닉상태와 셀상태는 훈련 과정에서는 사용하지 않는다(_)
decoder_outputs = decoder_dense(decoder_outputs)                                    # 출력층의 크기는 번역문의 글자(또는 단어)가 가질 수 있는 크기이다

In [ ]:
# 모델 훈련
# full data를 이용하여 epochs를 50회 정도 주어야 제대로 결과가 나온다.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=100, validation_split=0.2)

In [21]:
# 예측용 Encoder
# 입력된 문장을 인코더에 넣어서 은닉상태와 셀상태를 얻는다
# encoder_inputs, encoder_states는 훈련용에서 구성한 것을 사용한다
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [22]:
# 예측용 Decoder
# Decoder가 산출한 결과를 받는 상태 벡터를 정의
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 은닉상태와 셀상태를 받음
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [23]:
# word로부터 idx를 얻는 것을 idx로부터 word를 얻는 것으로 바꿈
index_to_src = dict((i, char) for char, i in word_index_source.items())
index_to_tar = dict((i, char) for char, i in word_index_target.items())
print(index_to_tar)

{1: ' ', 2: '\t', 3: '\n', 4: '.', 5: '어', 6: '이', 7: '톰', 8: '해', 9: '아', 10: '은', 11: '다', 12: '그', 13: '가', 14: '는', 15: '?', 16: '나', 17: '거', 18: '!', 19: '고', 20: '하', 21: '을', 22: '했', 23: '요', 24: '있', 25: '야', 26: '지', 27: '었', 28: '사', 29: '난', 30: '말', 31: '들', 32: '기', 33: '게', 34: '리', 35: '를', 36: '니', 37: '도', 38: '우', 39: '좋', 40: '와', 41: '내', 42: '에', 43: '람', 44: '무', 45: '자', 46: '마', 47: '서', 48: '봐', 49: '한', 50: '계', 51: '안', 52: '네', 53: '시', 54: '속', 55: '너', 56: '수', 57: '모', 58: '만', 59: '짓', 60: '라', 61: '두', 62: '누', 63: '일', 64: '세', 65: '정', 66: '웃', 67: '로', 68: '않', 69: '줘', 70: '았', 71: '제', 72: '렸', 73: '걸', 74: '없', 75: '려', 76: '물', 77: '미', 78: '저', 79: '여', 80: '건', 81: '죽', 82: '으', 83: '워', 84: '조', 85: '주', 86: '린', 87: '신', 88: '것', 89: '의', 90: '져', 91: '심', 92: '좀', 93: '운', 94: '인', 95: '구', 96: '진', 97: '렇', 98: '날', 99: '입', 100: '래', 101: '울', 102: '전', 103: '빨', 104: '피', 105: '히', 106: '할', 107: '보', 108: '러', 109: '파', 110: '소', 111: 

In [29]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)           # 입력문을 인코더에 넣어 문장의 상태 벡터를 얻는다    
    target_seq = np.zeros((1, 1, len(word_index_target)+1))   # 디코더를 초기화한다
    target_seq[0, 0, word_index_target['\t']] = 1.            # 디코더의 첫 시작은 <\t>로 시작하므로 \t 위치에 원-핫 인코딩으로 기록
    
    stop_condition = False
    decoded_sentence = ""
    
    #stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
      output_tokens, h, c = decoder_model.predict([target_seq] + states_value)  # target_input과 문장의 상태 벡터 입력      
      sampled_token_index = np.argmax(output_tokens)          # 가장 큰 값을 갖는 인덱스를 선택
      
      # sampled_token_index가 0이 나오면, index_to_tar 0은 없으므로 에러가 발생한다.
      # 이 경우 공백을 갖는 index 1로 치환한다.
      # 대량의 데이터에서는 확률상 0이 나오지 않고, 소량의 데이터에서는 대부분 0이 나온다.
      # 소량의 데이터에서 index_to_tar 0으로 치우치는 이유는, 0을 제외한 어떠한 것도 확률이 낮기 때문
      # 전체의 합을 1로 만들기 위해서, 빈 영역인 0번에 가장 높은 값이 들어가기 때문이다.
      # 즉, 0으로 나왔다는 것은 예측기가 어떠한 음절도 가능성이 낮은 것으로 본다 라고 해석할 수 있다
      if(sampled_token_index==0):  
        sampled_token_index = 1
        
      sampled_char = index_to_tar[sampled_token_index]
      decoded_sentence += sampled_char

      # 종료표시인 <\n>에 도달하면 중단.
      if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):  # 예측 종료 조건 (\n 또는 최대길이)
        stop_condition = True

      # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
      target_seq = np.zeros((1, 1, len(word_index_target)+1))   # target_input 초기화
      target_seq[0, 0, sampled_token_index] = 1.                # 직전 예측 결과를 sampled_token_index 위치에 원-핫 인코딩으로 기록

      # 상태를 업데이트 합니다.
      states_value = [h, c]

    return decoded_sentence

In [30]:
import numpy as np
for seq_index in [1, 2, 3]:                         # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]  # 3차원 배열에서는 이와 같이 [n:n+1] 형태로 해주어야 3차원이 유지되면서 n번째가 출력된다
  decoded_sentence = decode_sequence(input_seq)
  
  print(35 * "-")
  print('입력 문장:', data.source[seq_index])
  print('정답 문장:', data.target[seq_index][:len(data.target[seq_index])])
  print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])    # '\n'은 빼고 출력

1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Hi.
정답 문장: 안녕.
번역기가 번역한 문장: 안녕.
1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Run!
정답 문장: 뛰어!
번역기가 번역한 문장: 안녕!
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Run.
정답 문장: 뛰어.
번역기가 번역한 문장: 뛰어.


In [36]:
# 1개 문장 예측하기
# 전처리
input_seq_1 = tokenizer_source.texts_to_sequences([list('wait!')])
input_seq_1 = pad_sequences(input_seq_1, maxlen=max_src_len, padding='post')
input_seq_1 = to_categorical(input_seq_1, num_classes=len(word_index_source)+1)

# 예측하기 
decoded_sentence = decode_sequence(input_seq_1)
print(decoded_sentence)

1/1 [==============================] - 0s 15ms/step
안녕!

